In [2]:
from pyspark import SparkContext,SparkConf
import xgboost as xgb
import math
import sys
import json
import time
import numpy as np
import csv
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

In [2]:
'''
input_file_path = sys.argv[1]
input_test_file_path = sys.argv[2]
output_file_path = sys.argv[3]

'''


input_folder_path = "/Users/gopi/Desktop/Project/"
input_test_file_path = "/Users/gopi/Desktop/Assignment3/yelp_val_in.csv"
output_file_path = "/Users/gopi/Desktop/Project/output_1.csv"

In [3]:

#SparkContext.setSystemProperty('spark.executor.memory', '2g')
conf = SparkConf().setMaster("local[*]").setAppName("project").set("spark.executor.memory", "4g").set("spark.driver.memory", "4g").set("spark.ui.port", "27017")
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/10 14:55:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
yelp_categories = ["Active Life", "Arts & Entertainment", "Automotive", "Bars", "Beauty & Spas", "Education", "Event Planning & Services", 
"Financial Services", "Food", "Health & Medical", "Home Services", "Hotels & Travel", "Local Flavor", "Local Services", "Mass Media", 
"Nightlife", "Pets", "Professional Services", "Public Services & Government", "Real Estate", "Religious Organizations", "Restaurants", "Shopping"]

print(len(yelp_categories))
cat_map = {}
for i,cat in enumerate(yelp_categories):
    cat_map[cat] = i

23


In [5]:
print(yelp_categories[14])

Mass Media


In [22]:
analyzer= SentimentIntensityAnalyzer()
def filter_users(user_id) :
    return user_id in user_id_set

def filter_business(business_id):
    return business_id in business_id_set

def load_user_data(user):
    result = [user['user_id'], user['review_count'], user['average_stars'], user.get('fans',0), user.get('useful', 0),user.get('cool',0),user.get('funny',0), int(user.get('yelping_since')[2:4]), user.get('friends','').count(','), user.get('elite','').count(','), user.get('compliment_hot',0), user.get('compliment_photos',0),user.get('compliment_writer',0),user.get('compliment_funny',0),user.get('compliment_cool',0)]
    return tuple(result)

def load_business_data(business):
    result = [business['business_id'], business['review_count'], business['stars'], business.get('latitude', 0), business.get('longitude',0), business.get('is_open',0), photo_count_business.get(business['business_id'],0), business["neighborhood"], business['categories']]
    return tuple(result)

def get_final_data(data):
    global bus_nf_count
    global usr_nf_count
    global bus_in_count
    global usr_in_count
    time_1 = time.time()
    user_id = data[0]
    business_id = data[1]
    
    final_data = []
    if user_id in user_data:
        user_id_data = user_data[user_id]
        user_id_data = list(user_id_data)
    elif user_id in user_avg_from_train:
        user_id_data = [user_avg_from_train[user_id][0], user_avg_from_train[user_id][1], 0,0,0,0,0,0,0,0]
        usr_in_count += 1
    else:
        usr_nf_count += 1
        user_id_data = [0 for i in range(10)]
    
    if business_id in business_data:
        business_id_data = business_data[business_id]
    elif business_id in business_avg_from_train:
        business_id_data = [business_avg_from_train[business_id][0], business_avg_from_train[business_id][1]]
        empty = [0 for i in range(42)]
        business_id_data.extend(empty)
        bus_in_count += 1
    else :
        bus_nf_count += 1
        business_id_data = [0 for i in range(44)]
    
    if(len(business_id_data)) != 44:
        business_id_data = list(business_id_data)
        neighbourhood = business_id_data[len(business_id_data) -2]
        business_id_data[len(business_id_data) -2] = 1 if neighbourhood != None and neighbourhood != "" else 0
        categories = business_id_data[-1]
        if categories == None:
            categories = ''
        encoded_category = get_encoded_category(categories.split(","))
        business_id_data = business_id_data[:-1]
        business_id_data += tuple(encoded_category)
        #
        if business_id in tips_count_business:
            business_id_data.append(tips_count_business[business_id])
        else :
            business_id_data.append(0)
            
        if business_id in checkin_count_business:
            business_id_data.append(checkin_count_business[business_id])
        else:
            business_id_data.append(0)
        
        '''
        if business_id in photo_count_business:
            business_id_data.append(photo_count_business[business_id])
        else:
            business_id_data.append(0)
        '''
        if business_id in good_count:
            business_id_data.extend([good_count[business_id]])
        else:
            business_id_data.extend([0])
            
        if business_id in bad_count:
            business_id_data.extend([bad_count[business_id]])
        else:
            business_id_data.extend([0])
            
        if business_id in business_attr:
            business_id_data.extend(business_attr[business_id])
        else :
            business_id_data.extend([0 for i in range(38)])
            
        key = (user_id, business_id)
        if key in tips_sentiment:
            business_id_data.extend(tips_sentiment[key])
        else:
            business_id_data.extend([0,0,0,0])
        '''
        if business_id in business_sentiment:
            business_id_data.extend(business_sentiment[business_id])
        else:
            business_id_data.extend([0,0])
         
            
        ''' 
            
        '''
        if key in review_sentiment:
            business_id_data.extend(review_sentiment[key])
        else:
            business_id_data.extend([0,0,0,0])
        if key in review_good_count:
            business_id_data.append(review_good_count[key])
        else:
            business_id_data.append(0)
            
        if key in review_bad_count:
            business_id_data.append(review_bad_count[key])
        else:
            business_id_data.append(0)
        '''
        
    else:
        print("business not found!!")
    
    final_data.extend(user_id_data)
    final_data.extend(business_id_data)
    
    if len(data) == 3:
        final_data.append(data[-1])

    return final_data

def get_encoded_category(category_list):
    category_list = [item.lstrip() for item in category_list]
    final = []
    yelp_cat_set = set(category_list)
    for i in range(len(yelp_categories)):
        if yelp_categories[i] in yelp_cat_set:
            final.append(1)
        else:
            final.append(0)
    return final

def get_checkin_count(data):
    business_id = data['business_id']
    time = data['time']
    count = 0
    for i in time.values():
        count += i
    return (business_id, count)

def get_sentiment(data):
    global analyzer
    text = data[1]
    sentiment_dict = analyzer.polarity_scores(text)
    result = list(sentiment_dict.values())
    return (data[0],result)

def get_avg_sentiment(data):
    #print(data)
    _id = data[0]
    result = [0,0,0,0]
    for sentiment in data[1]:
        result[0] += sentiment[0]
        result[1] += sentiment[1]
        result[2] += sentiment[2]
        result[3] += sentiment[3]
        
    #result = [item/len(data[1]) for item in result]
    return (_id, result)

def compute_rmse(pred_data,data_type):

    file_2 = open("/Users/gopi/Desktop/Project/yelp_val.csv", "r")
    csv_2 = csv.reader(file_2, delimiter=",")
    pred_2 = []
    c = 0
    actual_range = {0:0,1:0,2:0,3:0,4:0}
    predict_range = {0:0,1:0,2:0,3:0,4:0}
    for row in csv_2:
        if c != 0:
            pred_2.append(float(row[2]))
        c+=1
            
    pred_1 = []
    if data_type == "CF":
        pred_1 = [data[2] for data in pred_data]
    else:
        pred_1 = pred_data
        
    total = 0
    for i in range(len(pred_1)):
        diff = abs(pred_1[i] - pred_2[i])
        if diff >= 0 and diff < 1:
            actual_range[0] += 1
        elif diff >= 1 and diff < 2:
            actual_range[1] += 1
        elif diff >= 2 and diff < 3:
            actual_range[2] += 1
        elif diff >= 3 and diff < 4:
            actual_range[3] += 1
        else:
            print(X_index[i][0], X_index[i][1])
            actual_range[4] += 1
        
        total += ((pred_1[i] - pred_2[i]) ** 2)
    
    print("actual range is:", actual_range)
    return math.sqrt(total/len(pred_1))

def compute_cat_rmse(pred_data,data_type):
    file_2 = open("/Users/gopi/Desktop/Project/yelp_val.csv", "r")
    csv_2 = csv.reader(file_2, delimiter=",")
    pred_2 = []
    ids = []
    c = 0
    for row in csv_2:
        if c != 0:
            pred_2.append(float(row[2]))
            ids.append(row[1])
        c+=1
            
    pred_1 = []
    if data_type == "CF":
        pred_1 = [data[2] for data in pred_data]
    else:
        pred_1 = pred_data
    
    rmse_list = [0 for i in range(23)]
    result = []
    for i in range(len(pred_1)):
        index = business_cat_data[ids[i]]
        rmse_list[index] += ((pred_1[i] - pred_2[i]) ** 2)
    for i in range(23):
        result.append(math.sqrt(rmse_list[i] / cat_counts[i]))
    return result
    
    
def get_category(categories):
    if categories == None:
        return len(yelp_categories)
    #print(categories)
    cat_list = categories.split(",")
    category = ''
    for cat in cat_list:
        if cat.lstrip() in yelp_categories:
            category = cat.lstrip()
            break
            
    index = len(yelp_categories) if category == '' else cat_map[category]
    return index

def count(business_id):
    return business_cat_data[business_id]

def get_attributes(data):
    if data == None:
        return [0 for i in range(38)]
    wifi = 1 if 'WiFi' in data and data['WiFi'] == 'free' else 0
    price_range = int(data['RestaurantsPriceRange2']) if 'RestaurantsPriceRange2' in data else 0
    credit_cards = 1 if 'BusinessAcceptsCreditCards' in data and bool(data['BusinessAcceptsCreditCards']) else 0
    good_for_kids = 1 if 'GoodForKids' in data and bool(data['GoodForKids']) == True else 0
    #caters = 1 if 'Caters' in data and bool(data['Caters']) else 0
    noise_level = data['NoiseLevel'] if 'NoiseLevel' in data else ''
    wh_acc = 1 if 'WheelchairAccessible' in data and bool(data['WheelchairAccessible']) == True else 0
    good_for_grps = 1 if 'RestaurantsGoodForGroups' in data and bool(data['RestaurantsGoodForGroups']) == True else 0
    wh_acc = 1 if 'WheelchairAccessible' in data and bool(data['WheelchairAccessible']) == True else 0
    res_takeout = 1 if 'RestaurantsTakeOut' in data and bool(data['RestaurantsTakeOut']) == True else 0
    res_res = 1 if 'RestaurantsReservations' in data and bool(data['RestaurantsReservations']) == True else 0
    res_del = 1 if 'RestaurantsDelivery' in data and bool(data['RestaurantsDelivery']) == True else 0
    by_app_only = 1 if 'ByAppointmentOnly' in data and bool(data['ByAppointmentOnly']) == True else 0
    bp = 1 if 'BikeParking' in data and bool(data['BikeParking']) == True else 0
    alcohol = 1 if 'Alcohol' in data and data['Alcohol'] != "none" else 0
    res_ts = 1 if 'RestaurantsTableService' in data and bool(data['RestaurantsTableService']) == True else 0
    has_tv = 1 if 'HasTV' in data and bool(data['HasTV']) else 0
    out_seating = 1 if 'OutdoorSeating' in data and bool(data['OutdoorSeating']) else 0
    res_attire = data['RestaurantsAttire'] if 'RestaurantsAttire' in data else ''
    if res_attire == 'casual':
        res_attire = 1
    elif res_attire == 'dressy':
        res_attire = 2
    elif res_attire == 'formal':
        res_attire = 3
    else:
        res_attire = 0
    #dogs_allowed = 1 if 'DogsAllowed' in data and bool(data['DogsAllowed']) else 0
    #accept_bc = 1 if 'BusinessAcceptsBitcoin' in data and bool(data['BusinessAcceptsBitcoin']) else 0
    #byob = 1 if 'BYOB' in data and bool(data['BYOB']) else 0
    #byob_cork = 1 if 'BYOBCorkage' in data and data['BYOBCorkage'] != "no" else 0
    
    g_f_m_map = data['GoodForMeal'] if 'GoodForMeal' in data else ''
    g_f_m = []
    if len(g_f_m_map) > 0:
        g_f_m_map = g_f_m_map.split(",")
        for value in g_f_m_map:
            if value != None and "True" in value:
                g_f_m.append(1)
            else:
                g_f_m.append(0)
    else:
        g_f_m = [0 for i in range(6)]
    if(len(g_f_m) < 6):
        for i in range(6-len(g_f_m)):
            g_f_m.append(0)
    elif len(g_f_m) > 6:
        g_f_m = g_f_m[:6]   
    
        
    bus_par_map = data['Ambience'] if 'Ambience' in data else ''
    bus_par = []
    if len(bus_par_map) > 0:
        bus_par_map = bus_par_map.split(",")
        for value in bus_par_map:
            if value != None and "True" in value:
                bus_par.append(1)
            else:
                bus_par.append(0)
    else:
        bus_par = [0 for i in range(8)]
    if(len(bus_par) < 8):
        for i in range(8-len(bus_par)):
            bus_par.append(0)
    elif len(bus_par) > 8:
        bus_par = bus_par[:8]
        
        
    best_nights_map = data['BestNights'] if 'BestNights' in data else ''
    best_nights = []
    if len(best_nights_map) > 0:
        best_nights_map = best_nights_map.split(",")
        for value in best_nights_map:
            if value != None and "True" in value:
                best_nights.append(1)
            else:
                best_nights.append(0)
    else:
        best_nights = [0 for i in range(7)]
        
    if(len(best_nights) < 7):
        for i in range(7-len(best_nights)):
            best_nights.append(0)
    elif len(best_nights) > 7:
        best_nights = best_nights[:7]

    
    if noise_level == "very_loud":
        noise_level = -1
    elif noise_level == 'loud':
        noise_level = 0
    elif noise_level == 'average':
        noise_level = 3
    else:
        noise_level = 5
        
    final = [wifi, price_range, credit_cards, good_for_kids, noise_level, wh_acc, good_for_grps, res_takeout, res_res, res_del, by_app_only, bp, alcohol, res_ts, has_tv, out_seating, res_attire]
    final.extend(g_f_m)
    final.extend(bus_par)
    final.extend(best_nights)
    return final

def write_data_to_file(output_file_path, data, predictions):
    file = open(output_file_path, 'w')
    file.write("user_id, business_id, prediction")
    file.write("\n")
    for i in range(len(data)):
        key = data[i]
        rating = predictions[i]
        output = key[0] + "," + key[1] + "," + str(rating)
        file.write(output)
        file.write("\n")
    file.close() 


In [7]:
#Model based predictions


initial_rdd = sc.textFile(input_folder_path + "review_train.json")
initial_rdd = initial_rdd.map(lambda line : json.loads(line)).map(lambda line: (line["user_id"], line["business_id"], float(line["stars"])))

photo_count_business = sc.textFile("photo.json").map(lambda x : json.loads(x)).map(lambda x : (x['business_id'], 1)).groupByKey().mapValues(list).map(lambda x : (x[0], sum(x[1]))).collectAsMap()
tips_count_business = sc.textFile("tip.json").map(lambda x : json.loads(x)).map(lambda x : (x['business_id'], 1)).groupByKey().mapValues(list).map(lambda x : (x[0], sum(x[1]))).collectAsMap()
checkin_count_business = sc.textFile("checkin.json").map(lambda x : json.loads(x)).map(lambda x : get_checkin_count(x)).groupByKey().mapValues(list).map(lambda x : (x[0], sum(x[1]))).collectAsMap()

#review_length_avg = sc.textFile("review_train.json").map(lambda x : json.loads(x)).map(lambda x : (x['user_id'], len(x['text']))).groupByKey().mapValues(list).map(lambda x : (x[0], sum(x[1]) / len(x[1]))).collectAsMap()

user_data = sc.textFile("user.json").map(lambda x : json.loads(x)).map(lambda user : load_user_data(user)).map(lambda x : (x[0],(x[1:]))).collectAsMap()
business_data = sc.textFile("business.json").map(lambda x : json.loads(x)).map(lambda user : load_business_data(user)).map(lambda x : (x[0],(x[1:]))).collectAsMap()
business_ids = set(business_data.keys())


/opt/homebrew/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling
/opt/homebrew/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling
/opt/homebrew/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling
/opt/homebrew/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling
/opt/homebrew/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling
/opt/homebrew/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with s

In [8]:
def get_good_count(data):
    good = 0
    bad = 0
    adj = ["amazing","good","great", "excellent", "super", "nice", "awesome", "better", "impressive", "fabulous", "authentic", "helpful", "friendly", "fantastic", "highly", "best","recommend", "delicious", "quality", "yum", "happy"]
    text = data[1].lower()
    for a in adj:
        if a in text:
            good += 1
    return (data[0],good)

def get_bad_count(data):
    bad = 0
    bad_adj = ["worst","bad", "terrible", "awful", "rude", "disgusting", "dread", "dreadful", "appaling", "lousy", "poor", "poorly", "sad", "stinky", "stinking", "stinks", "stink", "gross", "grossed", "defective", "disappointed", "flies"] 
    text = data[1].lower()
    for b in bad_adj:
        if b in text:
            bad += 1
    return (data[0],bad)

business_cat_data= sc.textFile("business.json").map(lambda x : json.loads(x)).map(lambda x : (x['business_id'], x['categories'])).mapValues(lambda x : get_category(x)).collectAsMap()
cat_counts = sc.textFile("review_train.json").map(lambda x : json.loads(x)).map(lambda x: (x['business_id'], x['user_id'])).map(lambda x : count(x[0])).map(lambda x : (x,1)).groupByKey().mapValues(list).mapValues(lambda x : sum(x)).sortBy(lambda x : x[0]).collectAsMap()
good_count = sc.textFile("tip.json").map(lambda x : json.loads(x)).map(lambda x : (x['business_id'], x['text'])).map(lambda x : get_good_count(x)).map(lambda x : (x[0], x[1])).groupByKey().mapValues(list).map(lambda x: (x[0], sum(x[1]))).collectAsMap()
bad_count = sc.textFile("tip.json").map(lambda x : json.loads(x)).map(lambda x : (x['business_id'], x['text'])).map(lambda x : get_bad_count(x)).map(lambda x : (x[0], x[1])).groupByKey().mapValues(list).map(lambda x: (x[0], sum(x[1]))).collectAsMap()
#review_good_count = sc.textFile("review_train.json").map(lambda x : json.loads(x)).map(lambda x : ((x['user_id'], x['business_id']), x['text'])).map(lambda x : get_good_count(x)).map(lambda x : (x[0], x[1])).groupByKey().mapValues(list).map(lambda x: (x[0], sum(x[1]))).collectAsMap()
#review_bad_count = sc.textFile("review_train.json").map(lambda x : json.loads(x)).map(lambda x : ((x['user_id'], x['business_id']), x['text'])).map(lambda x : get_bad_count(x)).map(lambda x : (x[0], x[1])).groupByKey().mapValues(list).map(lambda x: (x[0], sum(x[1]))).collectAsMap()



/opt/homebrew/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling


In [9]:
start = time.time()
tips_sentiment = sc.textFile("tip.json").map(lambda x : json.loads(x)).map(lambda x : ((x['user_id'], x['business_id']),x['text'])).map(lambda x: get_sentiment(x)).collectAsMap()
#tips_bus_sem = tips_sentiment.map(lambda x : (x[0][1], x[1])).groupByKey().mapValues(list).map(lambda x : get_avg_sentiment(x)).collectAsMap()
#tips_user_sem = tips_sentiment.map(lambda x : (x[0][0], x[1])).groupByKey().mapValues(list).map(lambda x : get_avg_sentiment(x)).collectAsMap()
#review_sentiment = sc.textFile("review_train.json").map(lambda x : json.loads(x)).map(lambda x : ((x['user_id'], x['business_id']), x['text'])).map(lambda x: get_sentiment(x)).collectAsMap()
print(time.time() - start)


14.149083137512207


In [23]:
business_attr = sc.textFile("business.json").map(lambda x : json.loads(x)).map(lambda x : (x['business_id'], x['attributes'])).mapValues(get_attributes).collectAsMap()
start_1 = time.time()
X_rdd = initial_rdd.filter(lambda x : x != None).map(lambda x : get_final_data(x))
#print(X_rdd.take(5))
X = X_rdd.map(lambda x : x[:-1]).collect()
Y = X_rdd.map(lambda x : x[-1]).collect()
testing_rdd = sc.textFile("/Users/gopi/Desktop/Project/yelp_val.csv")
testing_header = testing_rdd.first()
testing_data_map = testing_rdd.filter(lambda x : x!= testing_header).map(lambda line : line.split(",")).map(lambda line: ((line[0], line[1]),"")).collectAsMap()

X_test = []
X_index = []
business_id_set_test = set()
for key in list(testing_data_map.keys()):
    business_id_set_test.add(key[1])
    X_test.append(get_final_data(key))
    X_index.append(key)

'''
for key,value in user_business_rating_map.items():
    X.append(get_final_data(key))
    Y.append(value)
'''
    
print("Time taken to load X,Y is:", time.time() - start_1)


Time taken to load X,Y is: 308.7852668762207


In [10]:
rdd = sc.textFile("1.txt").filter(lambda x : 'hello' in x).collect()
rdd_1 = sc.textFile("country.csv").map(lambda x : x.split(",")).map(lambda x : (x[1],float(x[3]))).filter(lambda x : x[1] > 10000).groupByKey().mapValues(list).mapValues(lambda x : max(x)).collect()
print(rdd_1)

[('X', 400.0), (' Z', 200.0)]


In [24]:
xg_n_estimators = [250]
xg_learning_rate = [0.16]
xg_max_depth = [5]

for i in range(len(xg_n_estimators)):
    xgboost_model = xgb.XGBRegressor(n_estimators = xg_n_estimators[i], max_depth = xg_max_depth[i], learning_rate = xg_learning_rate[i])
    xgboost_model.fit(X,Y)
    predictions = xgboost_model.predict(X_test)
    
    for i in range(len(predictions)):
        if predictions[i] > 5.0:
            predictions[i] = 5.0
        if predictions[i] < 1:
            predictions[i] = 1.0

        
    #predictions[i] = math.floor(predictions[i])
    #pred_lr[i] = math.floor(pred_lr[i])

    print("xgboost rmse is:", compute_rmse(predictions, "model"))


dRLqHBA0Z-jnlTvWJ4aoZQ d0qktSyIFR6AlR9kyXqA8g
actual range is: {0: 102517, 1: 32695, 2: 6086, 3: 745, 4: 1}
xgboost rmse is: 0.9741062470639191


In [ ]:

write_data_to_file(output_file_path, X_index, predictions)

In [21]:
print("xgboost rmse is:", compute_rmse(predictions, "model"))
print(xgboost_model.feature_importances_)
print(xgboost_model.max_depth)

actual range is: {0: 102570, 1: 32633, 2: 6083, 3: 758, 4: 0}
xgboost rmse is: 0.9738056179259277
[0.00432289 0.0977509  0.00352983 0.00400111 0.00407389 0.00417507
 0.00366203 0.00408729 0.00386539 0.00461947 0.00531972 0.00592296
 0.00447216 0.         0.00371616 0.33984336 0.00486238 0.0056343
 0.00995975 0.00409073 0.00200409 0.00375942 0.00796523 0.00426353
 0.00360385 0.00408428 0.00601467 0.00410307 0.00424875 0.00420407
 0.00344584 0.00434967 0.00468272 0.00321691 0.00420305 0.
 0.00486347 0.00298845 0.00269764 0.00314097 0.0051004  0.00170246
 0.02376023 0.00340164 0.00420758 0.00532422 0.00438506 0.00471826
 0.00489518 0.01371484 0.00287112 0.00464894 0.00312182 0.00504023
 0.02215073 0.01798684 0.03262547 0.01747278 0.00394538 0.00607192
 0.01360019 0.01202593 0.00380454 0.05604648 0.00430456 0.00325057
 0.00994064 0.00309802 0.00375381 0.00459894 0.00467554 0.00203807
 0.00413311 0.00298644 0.00392999 0.00303682 0.00261134 0.00400372
 0.00433168 0.00330499 0.00295406 0.0037